In [3]:
import numpy as np
import pandas as pd
from datetime import *

In [196]:
scores = pd.read_csv('data/scores/scores{}.csv'.format(2013))[['Season','Week','HomeID','VisID','HomeConf','VisConf','HomeFinal','VisFinal','Spread','NeutralSite']]

In [188]:
def pythagoreanWinPct(teams, ind, exp):
    game = pd.Series([0.0 for x in range(len(ind))], index=ind)
    
    for i, team in enumerate(teams):
        pos = 'Home' if i == 0 else 'Vis'
        if (team[0] != 0) & (team[1] != 0):
            game['{}PythPct'.format(pos)] =  round((team[0]**exp)/(team[0]**exp + team[1]**exp), 2) #pyth pct
            game['{}PythWins'.format(pos)] = game['{}PythPct'.format(pos)] * team[2] #pyth wins
            game['{}PythPctDiff'.format(pos)] = (team[3]/team[2]) - game['{}PythPct'.format(pos)] #diff in pct
            game['{}PythWinsDiff'.format(pos)] = team[3] - game['{}PythWins'.format(pos)] #diff in wins
    return game


def pointsWinsGames(teams, week, scores, ind):
    
    game = pd.Series([0.0 for x in range(len(ind))], index=ind)
    
    for i, team in enumerate(teams):
        pos = 'Home' if i == 0 else 'Vis'
        prev_home, prev_vis = scores.loc[scores['HomeID'] == team].tail(1), scores.loc[scores['VisID'] == team].tail(1)
        if (prev_home.shape[0] == 0) & (prev_vis.shape[0] == 0):
            continue
        elif prev_home.shape[0] == 0:
            prev_game = prev_vis
            prev_pos, prev_opp = 'Vis', 'Home'
        elif prev_vis.shape[0] == 0:
            prev_game = prev_home
            prev_pos, prev_opp = 'Home','Vis' 
        elif int(prev_home['Week']) < int(prev_vis['Week']):
            prev_game = prev_vis
            prev_pos, prev_opp = 'Vis', 'Home'
        elif int(prev_vis['Week']) < int(prev_home['Week']):
            prev_game = prev_home
            prev_pos, prev_opp = 'Home','Vis' 
            
        game['{}PtsF'.format(pos)] = int(prev_game['{}PtsF'.format(prev_pos)]) + int(prev_game['{}Final'.format(prev_pos)])
        game['{}PtsA'.format(pos)] = int(prev_game['{}PtsA'.format(prev_pos)]) + int(prev_game['{}Final'.format(prev_opp)])
        
        if prev_pos == 'Home':
            game['{}Wins'.format(pos)] = int(prev_game['{}Wins'.format(prev_pos)]) + 1 if int(prev_game['Spread']) > 0 else int(prev_game['{}Wins'.format(prev_pos)])
        else:
            game['{}Wins'.format(pos)] = int(prev_game['{}Wins'.format(prev_pos)]) + 1 if int(prev_game['Spread']) < 0 else int(prev_game['{}Wins'.format(prev_pos)])
        
        game['{}Games'.format(pos)] = int(prev_game['{}Games'.format(prev_pos)]) + 1
        game['{}WinPct'.format(pos)] = round(int(game['{}Wins'.format(pos)])/int(game['{}Games'.format(pos)]), 2)
        
    return game



In [209]:
def meanRevert(team, prev_year, avg_rating, new_year, rating_interest):
    if prev_year['VisID'] == team:
        prev_rating = int(prev_year['Vis{}'.format(rating_interest)])
    elif prev_year['HomeID'] == team:
        prev_rating = int(prev_year['Home{}'.format(rating_interest)])
    else:
        prev_rating = 0
    return prev_rating - (1/3)*(prev_rating - avg_rating)

In [202]:
def addPointsWinsGames(year):
    scores = pd.read_csv('data/scores/scores{}.csv'.format(year))[['Season','Week','HomeID','VisID','HomeConf','VisConf','HomeFinal','VisFinal','Spread','NeutralSite']]
    #scores = scores.assign(HomePtsF=0,HomePtsA=0,HomeWins=0,HomeGames=0,HomeWinPct=0,VisPtsF=0,VisPtsA=0,VisWins=0,VisGames=0,VisWinPct=0)
    #ind = ['HomePtsF','HomePtsA', 'HomeWins','HomeGames','VisPtsF','VisPtsA', 'VisWins', 'VisGames']
    scores = scores.assign(HomePtsF=0,HomePtsA=0,HomeWins=0,HomeGames=0,HomeWinPct=0,VisPtsF=0,VisPtsA=0,VisWins=0,VisGames=0,VisWinPct=0)
    ind = ['HomePtsF','HomePtsA', 'HomeWins','HomeGames','HomeWinPct','VisPtsF','VisPtsA', 'VisWins', 'VisGames','VisWinPct']
    
    scores_list = [group for _, group in scores.groupby('Week')]
    
    for i, score in enumerate(scores_list[1:]):
        i += 1
        prev_weeks = pd.concat(scores_list[:i])
        scores_list[i][ind] = scores_list[i].apply(lambda x: pointsWinsGames((x['HomeID'],x['VisID']),x['Week'],prev_weeks, ind), axis = 1)
    scores = pd.concat(scores_list)
    
    return scores

def addPythWins(scores, exp):
    scores = scores.assign(HomePythPct=0,HomePythWins=0,VisPythPct=0,VisPythWins=0)
    ind = ['HomePythPct','HomePythWins','HomePythPctDiff','HomePythWinsDiff','VisPythPct','VisPythWins','VisPythPctDiff','VisPythWinsDiff']
    scores[ind] = scores.apply(lambda x: pythagoreanWinPct([(x['HomePtsF'],x['HomePtsA'],x['HomeGames'],x['HomeWins']),(x['VisPtsF'],x['VisPtsA'],x['VisGames'],x['VisWins'])],ind,exp),axis=1)
    return scores

In [203]:
scores_2013 = addPointsWinsGames(2013)
scores_2013 = addPythWins(scores_2013, 2.37)

In [201]:
scores_2013.tail()

KeyError: 'HomeConf'

In [61]:
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.metrics import *

In [71]:

for wins in ['Win','Pyth']:

    X_train, X_test, y_train, y_test = train_test_split(scores_2013[['Home{}Pct'.format(wins),'Vis{}Pct'.format(wins)]], scores_2013['Spread'], test_size=.2)
    lm = LinearRegression().fit(X_train, y_train)
    predictions = lm.predict(X_test)
    print(mean_squared_error(y_test, predictions))

577.2073312087921
407.171777209537
